In [71]:
import networkx as nx
import pandas as pd

data = pd.read_csv('C:/Users/timothy/Downloads/Camden_Licensing_Applications_Beta.csv'
                   ,parse_dates=['Received Date','Refused Date','Approved Date'])

data['Ward Name'] = data['Ward Name'].fillna(value='X')
data = data[np.isfinite(data['Latitude'])]
full_graph = nx.Graph()

full_graph.add_nodes_from(data['Application Reference'].unique(),typen='Application',color='black')
full_graph.add_nodes_from(data['Premises Name'].unique(),typen='Premise',color='blue')
full_graph.add_nodes_from(data['Application Legislation'].unique(),typen='Legislation',color='green')
full_graph.add_nodes_from(data['Ward Name'].unique(),typen='Ward',color='red')
full_graph.add_nodes_from(data['Application Status'].unique(),typen='Status',color='yellow')
full_graph.add_nodes_from(data['Application Type'].unique(),typen='Type',color='purple')
"""
col index to name
Application Reference	Application Type	Premises Name	Premises Address	Application Legislation	Application Status	Application Objections	Approved Date	Date Of Events	Expiry Date	Hearing Date	Last Date For Representations	Received Date	Refused Date	Cumulative Impact Policy Area Name	Ward Code	Ward Name	Easting	Northing	Longitude	Latitude	Spatial Accuracy	Last Uploaded	Location	Socrata ID	Organisation URI
0	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25
"""


#adding edges
for row in data.iterrows():
    full_graph.add_edge(row[1][2],row[1][0],name='made application',received=str(row[1][12]),year=row[1][12].year)
    full_graph.add_edge(row[1][0],row[1][16],name='application has ward')
    if(row[1][5]=='Application Rejected' or row[1][5]=='Application Rejected - Incomplete Documentation'):
        full_graph.add_edge(row[1][0],row[1][5],name='has status',refused=row[1][13],refusedyear=row[1][13].year)
    else:
        full_graph.add_edge(row[1][0],row[1][5],name='has status',approved=row[1][7],approvedyear=row[1][7].year)
    full_graph.add_edge(row[1][0],row[1][4],name='is of legislation type')
    full_graph.add_edge(row[1][0],row[1][1],name='is of type')
    


In [72]:

per_premise = data.groupby('Premises Name',as_index=False).min()
per_premise

,Premises Name,Application Reference,Application Type,Premises Address,Application Status,Application Objections,Approved Date,Date Of Events,Received Date,Refused Date,Ward Name,Easting,Northing,Longitude,Latitude,Spatial Accuracy,Last Uploaded,Location,Socrata ID,Organisation URI
0,001 The Stables Market,APP\LATE-TEN\001252,Late Temporary Event Notice,"THE STABLES MARKET CHALK FARM ROAD, LONDON, NW...",Application Approved,No,2015-01-06,NaN,2015-05-15,NaT,Camden Town with Primrose Hill,528518.62,184219.15,-0.148160,51.542256,Nearest OS Open Map-Local Building,04/05/2017 07:36:36 PM,"(51.542256, -0.14816)",32959,http://opendatacommunities.org/id/london-borou...
1,02 Forum,APP\PREMISES-NEW\001372,Premises - New Licence,"9 17 HIGHGATE ROAD, LONDON, NW5 1JY",Application Approved,No,2005-09-26,NaN,2005-07-29,2010-03-17,Kentish Town,528923.29,185300.87,-0.141932,51.551885,Nearest OS Open Map-Local Building,04/05/2017 07:36:28 PM,"(51.551885, -0.141932)",14668,http://opendatacommunities.org/id/london-borou...
2,06 St Chads Place,APP\TEN\002278,Temporary Event Notice,"6 ST CHAD'S PLACE, LONDON, WC1X 9HH",Application Approved,No,2008-04-24,NaN,2008-04-16,NaT,King's Cross,530530.86,182970.86,-0.119622,51.530577,Nearest OS Open Map-Local Building,04/05/2017 07:36:30 PM,"(51.530577, -0.119622)",18929,http://opendatacommunities.org/id/london-borou...
3,06 St Chads Place Bar &Restaurant,2009/2730/TC,Table and Chairs,"6 St Chad's Place, London, WC1X 9HH",Final Decision,0,NaT,NaN,NaT,NaT,King's Cross,530530.86,182970.86,-0.119622,51.530577,Nearest OS Open Map-Local Building,04/05/2017 07:36:37 PM,"(51.530577, -0.119622)",35490,http://opendatacommunities.org/id/london-borou...
4,1 Book Mews,APP\TEN\011649,Temporary Event Notice,"BOOK MEWS FLITCROFT STREET, LONDON, WC2H 8DJ",Application Approved,No,2015-10-07,NaN,2015-09-07,NaT,Holborn and Covent Garden,529900.15,181223.29,-0.129353,51.515018,Nearest OS Open Map-Local Building,04/05/2017 07:36:26 PM,"(51.515018, -0.129353)",10193,http://opendatacommunities.org/id/london-borou...
5,1 Earlham Street,APP\PREMISES-NEW\001610,Premises - New Licence,"1 EARLHAM STREET, LONDON, WC2H 9LL",Application Approved,No,2010-02-09,NaN,2010-01-06,2016-06-17,Holborn and Covent Garden,530005.14,181097.16,-0.127887,51.513860,Nearest OS Open Map-Local Building,04/05/2017 07:36:27 PM,"(51.51386, -0.127887)",13344,http://opendatacommunities.org/id/london-borou...
6,1 Kent House,APP\TEN\011497,Temporary Event Notice,"KENT HOUSE FERDINAND STREET, LONDON, NW1 8ET",Application Approved,No,2015-05-19,NaN,2015-05-17,NaT,Haverstock,528422.64,184397.92,-0.149478,51.543885,Nearest OS Open Map-Local Building,04/05/2017 07:36:26 PM,"(51.543885, -0.149478)",10331,http://opendatacommunities.org/id/london-borou...
7,1 Phoenix Road,APP\TEN\008262,Temporary Event Notice,"1A PHOENIX STREET, LONDON, WC2H 8BU",Application Approved,No,2012-01-08,NaN,2012-07-30,NaT,Holborn and Covent Garden,529907.57,181172.80,-0.129265,51.514562,Nearest OS Open Map-Local Building,04/05/2017 07:36:27 PM,"(51.514562, -0.129265)",13019,http://opendatacommunities.org/id/london-borou...
8,1 and 2 Neal's Yard,2011/2766/TC,Table and Chairs,"London, WC2H 9DP",Final Decision,0,2011-09-20,NaN,2011-06-14,NaT,Holborn and Covent Garden,530096.62,181164.76,-0.126545,51.514446,Nearest OS Open Map-Local Building,04/05/2017 07:36:37 PM,"(51.514446, -0.126545)",34991,http://opendatacommunities.org/id/london-borou...
9,10 Strathray Gardens,APP\TEN\013048,Temporary Event Notice,"10 STRATHRAY GARDENS, LONDON, NW3 4NY",Application Approved,No,2016-10-18,NaN,2016-10-16,NaT,Belsize,526993.50,184525.51,-0.170029,51.545354,Nearest OS Open Map-Local Building,04/05/2017 07:36:25 PM,"(51.545354, -0.170029)",8922,http://opendatacommunities.org/id/london-borou...


In [73]:
import numpy as np
def distance(long1, lat1,long2,lat2):
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    long1 = np.radians(long1)
    long2 = np.radians(long2)
    #earth radius
    R = 6371
    a = (np.sin((lat2-lat1)/2))**2 + np.cos(lat1) * np.cos(lat2) * (np.sin((long2-long1)/2))**2
    c = 2 * np.arctan2(np.sqrt(a),np.sqrt(1-a))
    d = R * c
    return d
   

In [74]:
max_dis_neighbor = 0.5
count = 0
neigbor_edges=0
for row in per_premise.iterrows():
    count +=1
    print(count/len(per_premise['Application Reference']),'%', end='\r', flush=True)
    for subrow in per_premise.iterrows():
        if row[1][0]==subrow[1][0]:
            pass
        else:
            dis = distance(row[1][13],row[1][14],subrow[1][13],subrow[1][14])
            if dis <max_dis_neighbor:
                if(full_graph.has_edge(row[1][0],subrow[1][0]) or full_graph.has_edge(subrow[1][0],row[1][0])):
                    pass
                else:
                    neigbor_edges +=1
                    full_graph.add_edge(row[1][0],subrow[1][0], name='is close to',edge_type='neighbor')
                

In [75]:
data['Received Date'] = pd.to_datetime(data['Received Date'],unit='D')
data['Receive Year'] = data['Received Date'].dt.year
data['Receive Year'] =data['Receive Year'].fillna(value=2005)
data['Receive Year'] = data['Receive Year'].astype(type('float', (float,), {}))
data['final decision'] = data['Approved Date']
data['final decision'] = data['final decision'].fillna(data['Refused Date'])
data['Decision Year'] = data['final decision'].dt.year
data['Decision Year'] = data['Decision Year'].astype(type('float', (float,), {}))
nx.set_node_attributes(full_graph,'Application Made'
                     ,dict(zip(data['Application Reference'], data['Receive Year'])))
nx.set_node_attributes(full_graph,'Application Close'
data

,Application Reference,Application Type,Premises Name,Premises Address,Application Legislation,Application Status,Application Objections,Approved Date,Date Of Events,Expiry Date,...,Easting,Northing,Longitude,Latitude,Spatial Accuracy,Last Uploaded,Location,Socrata ID,Organisation URI,final decision
0,GGN\00201,Gaming Notification,Secrets (Eversholt St),"34 38 EVERSHOLT STREET, LONDON, NW1 1DA",GA2005,Application Approved,No,NaT,NaN,NaN,...,529635.38,182806.00,-0.132585,51.529302,Nearest OS Open Map-Local Building,04/05/2017 07:36:25 PM,"(51.529302, -0.132585)",8158,http://opendatacommunities.org/id/london-borou...,NaT
1,APP\PREMISES-TRANS\000812,Premises Licence - Transfer Application,Sainath News,"2 EARLHAM STREET, LONDON, WC2H 9RY",LA2003,Application Approved In Full,No,2011-09-20,NaN,NaN,...,529985.02,181028.22,-0.128203,51.513245,Nearest OS Open Map-Local Building,04/05/2017 07:36:34 PM,"(51.513245, -0.128203)",27384,http://opendatacommunities.org/id/london-borou...,2011-09-20
2,APP\TEN\010414,Temporary Event Notice,Rosslyn Hill Chapel,"3 COTTAGE EXCLUDING BASEMENT PILGRIM'S PLACE, ...",LA2003,Application Approved,No,2014-02-06,NaN,NaN,...,526730.59,185621.08,-0.173424,51.555259,Nearest OS Open Map-Local Building,04/05/2017 07:36:26 PM,"(51.555259, -0.173424)",11305,http://opendatacommunities.org/id/london-borou...,2014-02-06
5,APP\PREMISES-VARYDPS\002281,Premises - Variation of DPS,Le Pain Quotidien,"64 TOTTENHAM COURT ROAD, LONDON, W1T 2ET",LA2003,Application Approved In Full,No,2012-09-01,NaN,NaN,...,529517.29,181805.45,-0.134654,51.520337,Nearest OS Open Map-Local Building,04/05/2017 07:36:32 PM,"(51.520337, -0.134654)",24117,http://opendatacommunities.org/id/london-borou...,2012-09-01
6,APP\TEN\008428,Temporary Event Notice,Bar Solo/Undersolo,"20 22 INVERNESS STREET, LONDON, NW1 7HJ",LA2003,Application Approved,No,2012-09-21,NaN,NaN,...,528778.86,183979.04,-0.144497,51.540039,Nearest OS Open Map-Local Building,04/05/2017 07:36:27 PM,"(51.540039, -0.144497)",13125,http://opendatacommunities.org/id/london-borou...,2012-09-21
8,APP\TEN\002280,Temporary Event Notice,06 St Chads Place,"6 ST CHAD'S PLACE, LONDON, WC1X 9HH",LA2003,Application Approved,No,2008-04-24,NaN,NaN,...,530530.86,182970.86,-0.119622,51.530577,Nearest OS Open Map-Local Building,04/05/2017 07:36:30 PM,"(51.530577, -0.119622)",18929,http://opendatacommunities.org/id/london-borou...,2008-04-24
9,APP\PREMISES-VARYDPS\003766,Premises - Variation of DPS,Belgo,"67 KINGSWAY, LONDON, WC2B 6TD",LA2003,Application Approved In Full,No,2015-10-21,NaN,NaN,...,530557.60,181249.67,-0.119874,51.515103,Nearest OS Open Map-Local Building,04/05/2017 07:36:31 PM,"(51.515103, -0.119874)",22731,http://opendatacommunities.org/id/london-borou...,2015-10-21
10,APP\TEN\011780,Temporary Event Notice,Chalk Farm Grill,"18 HAVERSTOCK HILL, LONDON, NW3 2BL",LA2003,Application Rejected,No,NaT,NaN,NaN,...,528190.18,184432.25,-0.152816,51.544246,Nearest OS Open Map-Local Building,04/05/2017 07:36:26 PM,"(51.544246, -0.152816)",10074,http://opendatacommunities.org/id/london-borou...,2015-05-09
11,APP\TEN\004351,Temporary Event Notice,White Horse,"154 156 FLEET ROAD, LONDON, NW3 2QX",LA2003,Application Approved,No,2009-01-12,NaN,NaN,...,527311.32,185537.58,-0.165083,51.554378,Nearest OS Open Map-Local Building,04/05/2017 07:36:29 PM,"(51.554378, -0.165083)",16961,http://opendatacommunities.org/id/london-borou...,2009-01-12
13,APP\TEN\012704,Temporary Event Notice,41 Brunswick Square,"41 BRUNSWICK SQUARE, LONDON, WC1N 1AU",LA2003,Application Approved,No,2016-05-06,NaN,NaN,...,530461.26,182399.00,-0.120837,51.525454,Nearest OS Open Map-Local Building,04/05/2017 07:36:25 PM,"(51.525454, -0.120837)",9230,http://opendatacommunities.org/id/london-borou...,2016-05-06


In [ ]:
nx.write_edgelist(full_graph,'full_graph3.0.txt',delimiter=';')
nx.write_gexf(full_graph,'full_thing.gexf')

In [ ]:
nx.write_edgelist(full_graph,'full_graph_time.txt',delimiter=';')
nx.write_gexf(full_graph,'full_thing_time.gefx')

In [366]:
all_nodes = full_graph.nodes(data=True)

nodes_df = pd.DataFrame(all_nodes,columns=['Node','Attributes'])
nodes_df.to_csv('all_nodes.csv')
